
<div style="border:1px solid black; padding:20px 20px;text-align: justify;text-justify: inter-word">
    <strong>Project - Motor Control 
    

<br/>


In [35]:
!pip install --upgrade tdmclient

In [36]:
!pip install tqdm scipy

In [37]:
!pip install numpy matplotlib pandas scipy tqdm

<strong>Imports :

In [1]:
import os
import sys
import math
from statistics import mean
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

import cv2
from vision import *
import importlib


In [56]:
import tdmclient.notebook
await tdmclient.notebook.start()

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
await tdmclient.notebook.stop()

In [ ]:
@onevent
def angle_to_rotate(pos_i, pos_f, theta_i):
    a = pos_f-pos_i
    theta_f = np.arctan2(a[1], a[0])
    r_theta = (theta_f*180)/np.pi-theta_i
    return r_theta
    
pos_thymio = np.array([1,1])
pos_goal = np.array([30,45])
theta = angle_to_rotate(pos_thymio,pos_goal,0)
print(theta)

56.61148642388849


In [ ]:
@onevent
def distance_to_cover(pos_i, pos_f):
    distance = np.linalg.norm(pos_f-pos_i)
    return distance

pos_thymio = np.array([0,0])
pos_goal = np.array([30,45])
theta = angle_to_rotate(pos_thymio,pos_goal,0)
dist = distance_to_cover(pos_thymio,pos_goal)
print(dist)

54.08326913195984


In [57]:
nb_node = 3
initial_orientation = 0 #degree
positions = np.array([[0,0],[30,45],[50,50],[17,15],[50,10]]);
rotations = np.zeros(positions.shape[0]-1)
distances =  np.zeros(positions.shape[0]-1)

rotations[0] = angle_to_rotate(positions[0],positions[1],initial_orientation)
orientation = initial_orientation+rotations[0]

for i in range(1,positions.shape[0]-1):
    rotations[i] = angle_to_rotate(positions[i],positions[i+1],orientation)
    distances[i-1] = distance_to_cover(positions[i-1],positions[i])
    orientation = orientation+rotations[i]

distances[distances.size-1]=distance_to_cover(positions[distances.size-1],positions[distances.size])

print('Rotations :',rotations)
print('Distances :',distances)

NameError: name 'angle_to_rotate' is not defined

In [ ]:
print(3537/121)

29.231404958677686


## Async implementation:

In [39]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 0102d6c0-0c6c-479f-8406-474092b0daba

In [59]:
await node.wait_for_variables()
node.var

NameError: name 'node' is not defined

In [2]:
def angle_to_rotate(pos_i, pos_f, theta_i):
    a = pos_f-pos_i
    theta_f = np.arctan2(a[1], a[0])
    r_theta = (theta_f*180)/np.pi-theta_i
    return r_theta

def distance_to_cover(pos_i, pos_f):
    distance = np.linalg.norm(pos_f-pos_i)
    return distance

In [3]:
def path(positions, initial_orientation):
    nb_node = 3
    rotations = np.zeros(positions.shape[0]-1)
    distances =  np.zeros(positions.shape[0]-1)
    
    rotations[0] = angle_to_rotate(positions[0],positions[1],initial_orientation)
    orientation = initial_orientation+rotations[0]
    
    for i in range(1,positions.shape[0]-1):
        rotations[i] = angle_to_rotate(positions[i],positions[i+1],orientation)
        distances[i-1] = distance_to_cover(positions[i-1],positions[i])
        orientation = orientation+rotations[i]

    distances[distances.size-1]=distance_to_cover(positions[distances.size-1],positions[distances.size])
    rotation_ratios = rotations/360
    return distances, rotations, orientation, rotation_ratios


<strong>Kalman filter : 

In [4]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

In [5]:
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import numpy as np
from tqdm import tqdm

<strong> Variance for position :

In [6]:
qpx = 0.04 # variance on position state
qpy = 0.04
rpx = 0.25 # variance on position measurement 
rpy = 0.25

qpx = 0.04 # variance on position state
qpy = 0.04 
rpx = 0.25 # variance on position measurement 
rpy = 0.25


<strong>Variance for angle : 

In [7]:
std_speed = 11.77
q_theta_nu = std_speed/2 # variance on speed state
r_theta_nu = std_speed/2 # variance on speed measurement 

print(q_theta_nu,r_theta_nu)

5.885 5.885


In [8]:
# Initialising the remaining constants
# units: length [mm], time [s]
A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
Q = np.array([[qpx, 0, 0], [0, qpy, 0],[0, 0, q_theta_nu]]); #w matrix
rotation_conv_factor = 0.41; #0.8 for 50 speed, 0.41 for 100 speed
speed_conv_factor_straight = 0.338; #straight
speed_conv_factor_left = 0.403;
speed_conv_factor_right = 0.425;

speed_conv_factor = 0.374 #mean of the two
transition_thresh = 500
l = 100 #mm, distance between the two wheels

def kalman_filter( x_est_prev, P_est_prev, Vr, Vl, Ts, camera_x=0, camera_y=0, camera_angle=0,):

    ## Prediciton through the a priori estimate
    # estimated mean of the state
    if abs(Vr-Vl) < 30:
        speed_conv_factor = speed_conv_factor_straight
    elif Vr<Vl:
        speed_conv_factor = speed_conv_factor_right
    else:
        speed_conv_factor = speed_conv_factor_left
    
    
    orientation = np.double(x_est_prev[2][0]%360)*2*np.pi/360
    #x_est_prev[2][0] = orientation
    B = np.array([[Ts*np.cos(orientation)/2,Ts*np.cos(orientation)/2],
         [Ts*np.sin(orientation)/2,Ts*np.sin(orientation)/2],
         [-Ts*360/(l*2*np.pi), Ts*360/(l*2*np.pi)]])
    U = np.array([[Vr*speed_conv_factor],[Vl*speed_conv_factor]])
    x_est_a_priori = np.dot(A, x_est_prev) + np.dot(B, U);
  
    
    # Estimated covariance of the state
    P_est_a_priori = np.dot(A, np.dot(P_est_prev, A.T));
    P_est_a_priori = P_est_a_priori + Q if type(Q) != type(None) else P_est_a_priori
    
    ## Update         
    # y, C, and R for a posteriori estimate, depending on transition
    if (camera_x != 0) : #XOR (one or the other but not both)
        # transition detected aka camera provides the position
        y = np.array([[camera_x],[camera_y],[camera_angle]])
        H = np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]])
        R = np.array([[rpx, 0, 0],[0, rpy,0],[0, 0, r_theta_nu]]) #not the value of the variance of the position measurments
        
        # innovation / measurement residual
        i = y - np.dot(H, x_est_a_priori); #measured-mean aka difference between the two states
        # measurement prediction covariance
        S = np.dot(H, np.dot(P_est_a_priori, H.T)) + R;
                 
        # Kalman gain (tells how much the predictions should be corrected based on the measurements)
        K = np.dot(P_est_a_priori, np.dot(H.T, np.linalg.inv(S)));
            
        # a posteriori estimate
        x_est = x_est_a_priori + np.dot(K,i);
        P_est = P_est_a_priori - np.dot(K,np.dot(H, P_est_a_priori));

    else:
        x_est = x_est_a_priori
        P_est = P_est_a_priori
     
    return x_est, P_est

Test of the filter

In [40]:
#test of the kalman filter
x_est = np.array([[0], [0], [0]]) # in mm !!!
#P_est = [1000 * np.ones(2)]

P_est = 0
Vr = 100;
Vl = 100;

Ts = 0.1

#camera_x = 4
#camera_y = 0
#camera_angle = 0

#for i in range(0,20):
new_x_est, new_P_est = kalman_filter(x_est, P_est, Vr, Vl, Ts, camera_x=0, camera_y=0, camera_angle=0);
x_est = new_x_est;
P_est = new_P_est;

print(x_est[2][0])
print("new_x_est\n",new_x_est,"\n new_P_est\n", new_P_est)

9.438649121200193e-17
new_x_est
 [[3.38000000e+00]
 [0.00000000e+00]
 [9.43864912e-17]] 
 new_P_est
 [[0.04  0.    0.   ]
 [0.    0.04  0.   ]
 [0.    0.    5.885]]


In [10]:
import time

In [11]:
def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }

In [12]:
import asyncio
import time
    
async def drive_for_seconds(client, seconds):
    # Set the motor speeds
    v = {
        "motor.left.target": [int(99)],
        "motor.right.target": [int(100)],
    }
    await node.set_variables(v)
    await asyncio.sleep(seconds)


async def turn_for_seconds(client, seconds,right):
    # Set the motor speeds
    if seconds != 0:
        if right==1:
            v = {
                "motor.left.target": [int(100)],
                "motor.right.target": [int(-100)],
            }
        else:
            v = {
                "motor.left.target": [int(-100)],
                "motor.right.target": [int(100)],
            }
        await node.set_variables(v)
    await asyncio.sleep(abs(seconds))

In [13]:
'''
vstop = {
    "motor.left.target": [int(0)],
    "motor.right.target": [int(0)],
}
await node.set_variables(vstop)
'''

'\nvstop = {\n    "motor.left.target": [int(0)],\n    "motor.right.target": [int(0)],\n}\nawait node.set_variables(vstop)\n'

In [14]:
def filter():#x_est, P_est, Ts):
    global x_est, P_est, Ts, read, angle, Vr, Vl

    thymio_data.append({"left_speed":node["motor.left.speed"],
                        "right_speed":node["motor.right.speed"]}) 

    #print(thymio_data[-1]["left_speed"])
    Vl = thymio_data[-1]["left_speed"]
    Vr = thymio_data[-1]["right_speed"]
    #print(Vr, Vl)

    x_est, P_est = kalman_filter(x_est, P_est, Vr, Vl, Ts, camera_x=0, camera_y=0, camera_angle=0);
    #print(x_est)
    

In [15]:
node.send_set_variables(motors(0, 0))


NameError: name 'node' is not defined

## Global navigation

In [16]:
#visibility graph

from collections import defaultdict
import numpy as np
import math
import heapq

class Point(object):
    __slots__ = ('x', 'y', 'polygon_id')

    def __init__(self, x, y, polygon_id=-1):
        self.x = float(x)
        self.y = float(y)
        self.polygon_id = polygon_id

    def __eq__(self, point):
        return point and self.x == point.x and self.y == point.y

    def __ne__(self, point):
        return not self.__eq__(point)

    def __lt__(self, point):
        """ This is only needed for shortest path calculations where heapq is
            used. When there are two points of equal distance, heapq will
            instead evaluate the Points, which doesnt work in Python 3 and
            throw a TypeError."""
        return hash(self) < hash(point)

    def __str__(self):
        return "(%.2f, %.2f)" % (self.x, self.y)

    def __hash__(self):
        return self.x.__hash__() ^ self.y.__hash__()

    def __repr__(self):
        return "Point(%.2f, %.2f)" % (self.x, self.y)

class Edge(object):
    __slots__ = ('p1', 'p2')

    def __init__(self, point1, point2):
        self.p1 = point1
        self.p2 = point2

    def get_adjacent(self, point):
        if point == self.p1:
            return self.p2
        return self.p1

    def __contains__(self, point):
        return self.p1 == point or self.p2 == point

    def __eq__(self, edge):
        if self.p1 == edge.p1 and self.p2 == edge.p2:
            return True
        if self.p1 == edge.p2 and self.p2 == edge.p1:
            return True
        return False

    def __ne__(self, edge):
        return not self.__eq__(edge)

    def __str__(self):
        return "({}, {})".format(self.p1, self.p2)

    def __repr__(self):
        return "Edge({!r}, {!r})".format(self.p1, self.p2)

    def __hash__(self):
        return self.p1.__hash__() ^ self.p2.__hash__()

class Graph(object):
    """
    A Graph is represented by a dict where the keys are Points in the Graph
    and the dict values are sets containing Edges incident on each Point.
    A separate set *edges* contains all Edges in the graph.

    The input must be a list of polygons, where each polygon is a list of
    in-order (clockwise or counter clockwise) Points. If only one polygon,
    it must still be a list in a list, i.e. [[Point(0,0), Point(2,0),
    Point(2,1)]].

    *polygons* dictionary: key is a integer polygon ID and values are the
    edges that make up the polygon. Note only polygons with 3 or more Points
    will be classified as a polygon. Non-polygons like just one Point will be
    given a polygon ID of -1 and not maintained in the dict.
    """

    def __init__(self, polygons):
        self.graph = defaultdict(set)
        self.edges = set()
        self.polygons = defaultdict(set)
        pid = 0
        for polygon in polygons:
            if polygon[0] == polygon[-1] and len(polygon) > 1:
                polygon.pop()
            for i, point in enumerate(polygon):
                sibling_point = polygon[(i + 1) % len(polygon)]
                edge = Edge(point, sibling_point)
                if len(polygon) > 2:
                    point.polygon_id = pid
                    sibling_point.polygon_id = pid
                    self.polygons[pid].add(edge)
                self.add_edge(edge)
            if len(polygon) > 2:
                pid += 1

    def get_adjacent_points(self, point):
        return [edge.get_adjacent(point) for edge in self[point]]

    def get_points(self):
        return list(self.graph)

    def get_edges(self):
        return self.edges
    
    def get_polygons(self):
        return self.polygons

    def add_edge(self, edge):
        self.graph[edge.p1].add(edge)
        self.graph[edge.p2].add(edge)
        self.edges.add(edge)

    def __contains__(self, item):
        if isinstance(item, Point):
            return item in self.graph
        if isinstance(item, Edge):
            return item in self.edges
        return False

    def __getitem__(self, point):
        if point in self.graph:
            return self.graph[point]
        return set()

    def __str__(self):
        res = ""
        for point in self.graph:
            res += "\n" + str(point) + ": "
            for edge in self.graph[point]:
                res += str(edge)
        return res

    def __repr__(self):
        return self.__str__()

In [17]:
# 0 - collinear
# 1 - clockwise
# -1 - counterclockwise

def orientation(p1, p2, p3):
    angle = (float)(p2.y - p1.y)*(p3.x-p2.x) - (float)(p3.y-p2.y)*(p2.x-p1.x)
    if(angle < 0):
        return -1
    if angle > 0:
        return 1
    return 0

In [18]:
def onSegment(p, q, r): 
    if ( (q.x <= max(p.x, r.x)) and (q.x >= min(p.x, r.x)) and 
           (q.y <= max(p.y, r.y)) and (q.y >= min(p.y, r.y))): 
        return True
    return False

In [19]:
def intersect(edge, obstacle):
    if(obstacle.p1 in edge) or (obstacle.p2 in edge):
        return False
    o1 = orientation(edge.p1, edge.p2, obstacle.p1)
    o2 = orientation(edge.p1, edge.p2, obstacle.p2)
    o3 = orientation(obstacle.p1, obstacle.p2, edge.p1)
    o4 = orientation(obstacle.p1, obstacle.p2, edge.p2)
    #general case 
    if (o1 != o2) and (o3 != o4):
        return True
    if (o1 == 0) and onSegment(edge.p1, obstacle.p1, edge.p2):
        return True
    if (o2 == 0) and onSegment(edge.p1, obstacle.p2, edge.p2):
        return True
    if (o3 == 0) and onSegment(obstacle.p1, obstacle.p2, edge.p1):
        return True
    if (o4 == 0) and onSegment(obstacle.p1, obstacle.p2, edge.p2):
        return True
    return False

In [20]:
def edge_in_polygon(p1, p2, graph):
    if p1.polygon_id != p2.polygon_id:
        return False
    if p1.polygon_id == -1 or p2.polygon_id == -1:
        return False
    #assumption that the obstacles are convex
    return True

In [21]:
def visible_vertices(point, graph, start = None, goal = None):
    obstacles = []
    for polygons in graph.get_polygons().values():
        obstacles += polygons
    points = graph.get_points()
    if start:
        points.append(start)
    if goal:
        points.append(goal)
    visible = []
    for other_point in points:
        if point == other_point: #looping over the other n-1 vertcies, assuming that n is the total number of vertices
            continue
        is_visible = False
        edge = Edge(point, other_point)
        if edge in graph: #if it is already part of the graph, it's either an edge of the obstacle, since it s enlarged we can traverse it, and if it's not it was obtained when checking the edge when the roles of points were reversed    
            continue
        intersects = False
        for obstacle in obstacles:
            if(intersect(edge, obstacle)):
                intersects = True
                break
        if intersects != False:
            continue
        #check if the visible edge is interior to its polygon
        if other_point not in graph.get_adjacent_points(point):
            if edge_in_polygon(point, other_point, graph):
                continue
        visible.append(other_point)
    return visible

In [22]:
#naive visibility graph
def build_graph(graph, start = None, goal = None):
    obstacles = []
    for polygons in graph.get_polygons().values():
        obstacles += polygons
    points = graph.get_points()
    if start:
        points.append(start)
    if goal:
        points.append(goal)
    for point in points:
        visible = visible_vertices(point, graph, start, goal)
        for p in visible:
            graph.add_edge(Edge(point, p))
    return graph

In [23]:
def distanceBetweenPoints(p1, p2):
    return math.sqrt((p2.x - p1.x)**2 + (p2.y - p1.y)**2)
def heuristic(point, goal):
    return distanceBetweenPoints(point, goal)

In [24]:
def Astar(graph, start, goal):
    #check if start and goal are valid points !!
    start_exists = start in graph
    goal_exists = goal in graph
    add_to_graph = Graph([])
    if not start_exists:
        for v in visible_vertices(start, graph):
            add_to_graph.add_edge(Edge(start, v))
    if not goal_exists:
        for v in visible_vertices(goal, graph):
            add_to_graph.add_edge(Edge(goal, v))
    points = graph.get_points()
    opened = [(heuristic(start,goal),start)]
    heapq.heapify(opened)
    closed = []
    cameFrom = dict()
    h = dict()
    costs = dict(zip(points, [np.inf for x in range(len(points))]))
    costs[start] = 0
    cameFrom = dict()
    nodes = []
    while len(opened) != 0:
        currentEstimate, current = heapq.heappop(opened)
        closed.append(current)
        if current == goal:
            while current != start:
                nodes.append(current)
                temp = cameFrom[current]
                current = temp
            nodes.append(current)
            return nodes, closed
        for neighbour in (graph.get_adjacent_points(current) + add_to_graph.get_adjacent_points(current)):
            cost = costs[current] + distanceBetweenPoints(current, neighbour)
            if neighbour in costs:
                if cost >= costs[neighbour]:
                    continue
            opened.append((cost + heuristic(neighbour, goal), neighbour))
            cameFrom[neighbour] = current
            costs[neighbour] = cost           
    print("No path found to goal")
    return nodes, closed


def global_navigation(polys, start, goal):

    graph = Graph(polys)
    build_graph(graph)
    
    nodes, closed = Astar(graph, start, goal)
    
    #for node in nodes:
    nodes = nodes[::-1]
    coordinates = np.array(nodes)
    print(coordinates)
    
    points = np.zeros((len(coordinates),2))
    
    for i in range(len(coordinates)):
        points[i]= [coordinates[i].x,coordinates[i].y]
        
    print(points*2) 
    return points
'''Change the Scale !!!!'''

'Change the Scale !!!!'

## Local avoidance

In [25]:
def test_saw_obstacle(threshold, verbose=True):
    print("\t Test saw obstacle: ")
    if any([x > threshold for x in node['prox.horizontal'][0:5]]):
        if verbose: print("\t\t Obstacle detected")
        return True
    
    return False

In [26]:
async def path_following(motor_speed=100, threshold=0,  verbose=False):
    
    global n, positions, x_est, P_est, Ts, read, Vr, Vl, n

    saw_obstacle = False
    
    if verbose: print("\t Path following ")
    await node.set_variables(motors(motor_speed, motor_speed))
           
    prev_state="local"
    
    while not saw_obstacle:
        
        if test_saw_obstacle(threshold, verbose=False):
            if prev_state=="local": 
                prev_state="global"
            saw_obstacle = True
            await client.sleep(0.1) #otherwise, variables would not be updated
        else:
            start = time.time()
            if (positions[n][0]+position_threshold > x_est[0][0]) and (positions[n][0]-position_threshold < x_est[0][0]):
                if (positions[n][1]+position_threshold > x_est[1][0]) and (positions[n][1]-position_threshold < x_est[1][0]):
                    n+=1
            angle = angle_to_rotate([x_est[0][0],x_est[1][0]],positions[n],x_est[2][0]);
            if (abs(angle)<angle_threshold):
                #Vr = 100
                #Vl = 100
                drive_task = asyncio.create_task(drive_for_seconds(client, Ts))
                await drive_task
    
            else:
                if angle>0:
                    right = 1
                    #Vr = -100
                    #Vl = 100
                else:
                    right = 0
                    #Vr = 100
                    #Vl = -100

                turn_task = asyncio.create_task(turn_for_seconds(client, Ts, right))
                await turn_task
            #print(x_est)
            stop = time.time()
            #print("time elapsed : ", start - stop)
            
    return 

In [27]:
async def local_avoidance(motor_speed=100, threshold=0, verbose=False):
    obstacleSpeedGain = [6, 4, -2, -6, -8]
    saw_obstacle = True
    speedL = motor_speed
    speedR = motor_speed
    prev_state = "global"

    if verbose:
        print("Local avoidance")

    while saw_obstacle:
        if test_saw_obstacle(threshold, verbose=False):
            if prev_state == "global":
                prev_state = "local"
                print("\t Transitioning to local avoidance")

            for i in range(5):
                speedL += node["prox.horizontal"][i] * obstacleSpeedGain[i] // 100
                speedR += node["prox.horizontal"][i] * obstacleSpeedGain[4 - i] // 100

          
            await node.set_variables(motors(speedL, speedR))
            if verbose:
                print("\t\t Adjusting for obstacle")

        else:
            saw_obstacle = False
            if prev_state == "local":
                #prev_state = "global"
                print(prev_state)
                print("\t Transitioning to global navigation")
                
               

            # Reset motor speeds when no obstacles are detected
            speedL = motor_speed
            speedR = motor_speed
            await node.set_variables(motors(speedL, speedR))
            
            # Add a small delay to avoid continuously checking for obstacles too quickly
            await client.sleep(0.1)




In [28]:
thymio_data = []
initial_orientation = 0 #degree

# Calculation of the fastest path # To get from the vision
polys = [[Point(170, 599), Point(170,731), Point(291,731), Point(291, 599)], [Point(431, 432), Point(398,489), Point(431, 550), Point(501, 550), Point(535, 491), Point(503, 435)], [Point(78, 129), Point(78,420), Point(531, 420), Point(531,129)]]
start = Point(76, 81)
goal = Point(498, 762)

positions = global_navigation(polys, start, goal)

positions = np.array([[0,0],[200,100],[10,30]])#[0,0],[30,45],[50,50],[17,15],[50,10]
angle_threshold = 5 #degrees
x_est = np.array([[positions[0][0]], [positions[0][1]], [initial_orientation]])
P_est = 0
Ts = 0.1
count = 0
read = 1
Vr = 0
Vl = 0
position_threshold = 10

n = 1
await node.wait_for_variables() # wait for Thymio variables values
rt = RepeatedTimer(Ts, filter)#, x_est, P_est, Ts)
try:
    print("entered try")
    for i in range(10):
        await path_following(verbose=True)
        await local_avoidance(verbose=True)
    #node.send_set_variables(motors(0, 0))
finally:
    rt.stop()
    node.send_set_variables(motors(0, 0))
    

[Point(76.00, 81.00) Point(78.00, 420.00) Point(498.00, 762.00)]
[[ 152.  162.]
 [ 156.  840.]
 [ 996. 1524.]]


NameError: name 'node' is not defined

In [29]:
await node.set_variables(motors(0, 0))

NameError: name 'node' is not defined

## Main

In [43]:
thymio_data = []
initial_orientation = 0 #degree

## ----- alexei -----
## ----- ------ -----

# Open the video capture
# cap = cv2.VideoCapture(0)
# # Read a frame from the video capture
# ret, frame = cap.read()

# # Applying vision functions
# thymio_result = locate_thymio(frame)
# obstacles_result = locate_static_obstacles(frame, 0)
# goal_result = locate_goal(frame)  
# table_origin_result = locate_table_origin(frame)

# start = Point(thymio_result[0][0][0], thymio_result[0][0][1])
# goal = Point(goal_result[0], goal_result[1])
# table_origin = [table_origin_result[0], table_origin_result[1]]
# polys = obstacles_result[2]
# polys = []
# for obstacle in obstacles_result[2]:
#     polygon = []
#     for corner in obstacle:
#         polygon.append(Point(corner[0], corner[1]))
#     polys.append(polygon)

# print("polys:\n",polys)

# polys = [[Point(855.00, 457.00), Point(1291.00, 457.00), Point(1291.00, 794.00), Point(855.00, 794.00)]]

## ----- ------ -----
## ----- ------ -----

# # Calculation of the fastest path # To get from the vision
# polys = [[Point(170, 599), Point(170,731), Point(291,731), Point(291, 599)], [Point(431, 432), Point(398,489), Point(431, 550), Point(501, 550), Point(535, 491), Point(503, 435)], [Point(78, 129), Point(78,420), Point(531, 420), Point(531,129)]]
# start = Point(76, 81)
# goal = Point(498, 762)

positions = global_navigation(polys, start, goal) 

print("--- positions nodes ----")
print(positions)
print("-------")

print(goal)

# # positions = np.array([[0,0],[200,100],[10,30]])#[0,0],[30,45],[50,50],[17,15],[50,10]
positions = np.array([[337.42857143, 214], [1291, 457], [1744,992]])
angle_threshold = 5 #degrees
x_est = np.array([[positions[0][0]], [positions[0][1]], [initial_orientation]])
P_est = 0
Ts = 0.1
count = 0
read = 1
Vr = 0
Vl = 0
position_threshold = 10

n = 1
await node.wait_for_variables() # wait for Thymio variables values
rt = RepeatedTimer(Ts, filter)#, x_est, P_est, Ts)

async def main():    

    global x_est, P_est, Ts, read, Vr, Vl, n
   
    
    try:
        for i in range(0,1000):
            start = time.time()
            if (positions[n][0]+position_threshold > x_est[0][0]) and (positions[n][0]-position_threshold < x_est[0][0]):
                if (positions[n][1]+position_threshold > x_est[1][0]) and (positions[n][1]-position_threshold < x_est[1][0]):
                    n+=1
            angle = angle_to_rotate([x_est[0][0],x_est[1][0]],positions[n],x_est[2][0]);
            if (abs(angle)<angle_threshold):
                #Vr = 100
                #Vl = 100
                drive_task = asyncio.create_task(drive_for_seconds(client, Ts))
                await drive_task
    
            else:
                if angle>0:
                    right = 1
                    #Vr = -100
                    #Vl = 100
                else:
                    right = 0
                    #Vr = 100
                    #Vl = -100

                turn_task = asyncio.create_task(turn_for_seconds(client, Ts, right))
                await turn_task
            #print(x_est)
            stop = time.time()
            print("time elapsed : ", start - stop)
    finally:
        rt.stop()
        node.send_set_variables(motors(0, 0))

await main()

[Point(337.43, 214.00) Point(1291.00, 457.00) Point(1744.00, 992.00)]
[[ 674.85714286  428.        ]
 [2582.          914.        ]
 [3488.         1984.        ]]
--- positions nodes ----
[[ 337.42857143  214.        ]
 [1291.          457.        ]
 [1744.          992.        ]]
-------
(1744.00, 992.00)
time elapsed :  -0.25577497482299805
time elapsed :  -0.20648598670959473
time elapsed :  -0.20187807083129883
time elapsed :  -0.20655488967895508
time elapsed :  -0.20770907402038574
time elapsed :  -0.20392107963562012
time elapsed :  -0.20760822296142578
time elapsed :  -0.20731210708618164
time elapsed :  -0.21071600914001465
time elapsed :  -0.20667600631713867
time elapsed :  -0.20823001861572266
time elapsed :  -0.20851516723632812
time elapsed :  -0.20895004272460938
time elapsed :  -0.20943212509155273
time elapsed :  -0.2082970142364502
time elapsed :  -0.20857501029968262
time elapsed :  -0.20671892166137695
time elapsed :  -0.20867705345153809
time elapsed :  -0.2081601

Exception in thread Exception in thread Thread-5:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
Thread-4:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/Users/alexei.ermochkine/Desktop/ma3/mobile_robotics/venv4mobilerobotics/lib/python3.11/site-packages/tdmclient/tcp.py", line 73, in run
    self.run()
  File "/Users/alexei.ermochkine/Desktop/ma3/mobile_robotics/venv4mobilerobotics/lib/python3.11/site-packages/tdmclient/tcp.py", line 73, in run
    packet = self.read_packet()
             ^^^^^^^^^^^^^^^^^^
  File "/Users/alexei.ermochkine/Desktop/ma3/mobile_robotics/venv4mobilerobotics/lib/python3.11/site-packages/tdmclient/tcp.py", line 66, in read_packet
    raise error
  File "/Users/alexei.ermochkine/Desktop/ma3/mobile_robotics/venv4mobilerobot

In [31]:
# Open the video capture
cap = cv2.VideoCapture(0)
# Read a frame from the video capture
ret, frame = cap.read()

obstacles_result = locate_static_obstacles(frame, 0)


434 333
387 1080
490 1080


/Users/alexei.ermochkine/Desktop/ma3/mobile_robotics/venv4mobilerobotics/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
%%refresh

UsageError: Cell magic `%%refresh` not found.
